In [1]:
import tensorflow as tf
tf.enable_eager_execution()

2024-05-02 19:29:41.188092: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import os
import random

# Function to load images and corresponding labels
def load_images_and_labels(folder_path):
    images = []
    labels = []
    label_names = []

    # Iterate over folders (labels)
    for label_folder in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label_folder)

        # Skip if it's not a directory
        if not os.path.isdir(label_path):
            continue

        # Append label name to the list
        label_names.append(label_folder)

        # Load images from the label folder
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            # Append image path to the list
            images.append(image_path)
            # Append label index to the labels list
            labels.append(len(label_names) - 1)  # Index of label name

    return images, labels, label_names

# Load images and labels
vggface2_folder = "Downloads/val"
vggface2_folder1="Downloads/test"
images, labels, label_names = load_images_and_labels(vggface2_folder)
images1,labels1,label_names1=load_images_and_labels(vggface2_folder1)
images=images+images1
labels=labels+labels1
label_names=label_names1


# Split data into train and test sets (adjust the split ratio as needed)
split_ratio = 0.8
num_images = len(images)
num_train = int(num_images * split_ratio)
random.seed(42)
train_indices = random.sample(range(num_images), num_train)
test_indices = list(set(range(num_images)) - set(train_indices))

# Write image paths and labels into train.txt and test.txt files
def write_to_file(file_path, image_paths, image_labels):
    with open(file_path, 'w') as file:
        for path, label in zip(image_paths, image_labels):
            file.write(f"{path} {label}\n")

write_to_file("Downloads/vgg_train.txt", [images[i] for i in train_indices], [labels[i] for i in train_indices])
write_to_file("Downloads/vgg_test.txt", [images[i] for i in test_indices], [labels[i] for i in test_indices])

print("Files train.txt and test.txt generated successfully.")


Files train.txt and test.txt generated successfully.


In [3]:

weight_decay = 1e-4
epoch_num = 50
hash_bit = 512
num_class = len(set(labels))
input_shape = (224, 224, 3)
label_smoothing = 0.1

# training config
batch_size = 128
train_num = len(train_indices) #6149
print(train_num)
iterations_per_epoch = int(train_num / batch_size) + 1
warm_iterations = iterations_per_epoch
print(iterations_per_epoch)


initial_learning_rate = 0.05
minimum_learning_rate = 0.0001

# test config
test_batch_size = 128
test_num = len(test_indices)
print(test_num)
test_iterations = int(test_num / test_batch_size) + 1

log_file = 'result/vggface2/vgg.txt'

# lr
learning_rate = [0.1, 0.01, 0.001]
boundaries = [80 * iterations_per_epoch, 120 * iterations_per_epoch]


# path
train_data_path = ''
test_data_path = ''
imagenet_train_path = 'Downloads/vgg_train.txt'
imagenet_test_path = 'Downloads/vgg_test.txt'


short_side_scale = (256, 384)
aspect_ratio_scale = (0.8, 1.25)
hue_delta = (-36, 36)
saturation_scale = (0.6, 1.4)
brightness_scale = (0.6, 1.4)
pca_std = 0.1

mean = [103.939, 116.779, 123.68]
std = [58.393, 57.12, 57.375]
eigval = [55.46, 4.794, 1.148]
eigvec = [[-0.5836, -0.6948, 0.4203],
          [-0.5808, -0.0045, -0.8140],
          [-0.5675, 0.7192, 0.4009]]


158154
1236
39539


In [4]:
!pip install opencv-python

import cv2
import numpy as np

"""
|------|
|      | height, y
|      |
|------|
 width, x
"""

def random_size(image, target_size=None):
    height, width, _ = np.shape(image)
    if target_size is None:
        # for test
        # target size is fixed
        target_size = np.random.randint(*short_side_scale)
    if height < width:
        size_ratio = target_size / height
    else:
        size_ratio = target_size / width
    resize_shape = (int(width * size_ratio), int(height * size_ratio))  # width and height in cv2 are opposite to np.shape()
    return cv2.resize(image, resize_shape)

def random_aspect(image):
    height, width, _ = np.shape(image)
    aspect_ratio = np.random.uniform(*aspect_ratio_scale)
    if height < width:
        resize_shape = (int(width * aspect_ratio), height)
    else:
        resize_shape = (width, int(height * aspect_ratio))
    return cv2.resize(image, resize_shape)

def random_crop(image):
    height, width, _ = np.shape(image)
    input_height, input_width, _ = input_shape
    crop_x = np.random.randint(0, width - input_width)
    crop_y = np.random.randint(0, height - input_height)
    return image[crop_y: crop_y + input_height, crop_x: crop_x + input_width, :]

def random_flip(image):
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
    return image

def random_hsv(image):
    random_h = np.random.uniform(*hue_delta)
    random_s = np.random.uniform(*saturation_scale)
    random_v = np.random.uniform(*brightness_scale)

    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image_hsv[:, :, 0] = image_hsv[:, :, 0] + random_h % 360.0  # hue
    image_hsv[:, :, 1] = np.minimum(image_hsv[:, :, 1] * random_s, 1.0)  # saturation
    image_hsv[:, :, 2] = np.minimum(image_hsv[:, :, 2] * random_v, 255.0)  # brightness

    return cv2.cvtColor(image_hsv, cv2.COLOR_HSV2BGR)

def random_pca(image):
    alpha = np.random.normal(0, pca_std, size=(3,))
    offset = np.dot(eigvec * alpha, eigval)
    image = image + offset
    return np.maximum(np.minimum(image, 255.0), 0.0)

def normalize(image):
    for i in range(3):
        image[..., i] = (image[..., i] - mean[i]) / std[i]
    return image

def center_crop(image):
    height, width, _ = np.shape(image)
    input_height, input_width, _ = input_shape
    crop_x = (width - input_width) // 2
    crop_y = (height - input_height) // 2
    return image[crop_y: crop_y + input_height, crop_x: crop_x + input_width, :]

def test_10_crop(image):
    """
    Standard 10 crop for test.
    Crop 4 corner and 1 center.
    Then flip it.
    """
    height, width, _ = np.shape(image)
    input_height, input_width, _ = input_shape
    center_crop_x = (width - input_width) // 2
    center_crop_y = (height - input_height) // 2

    images = []
    images.append(image[:input_height, :input_width, :])  # left top
    images.append(image[:input_height, -input_width:, :])  # right top
    images.append(image[-input_height:, :input_width, :])  # left bottom
    images.append(image[-input_height:, -input_width:, :])  # right bottom
    images.append(image[center_crop_y: center_crop_y + input_height, center_crop_x: center_crop_x + input_width, :])

    image = cv2.flip(image, 1)
    images.append(image[:input_height, :input_width, :])  # left top
    images.append(image[:input_height, -input_width:, :])  # right top
    images.append(image[-input_height:, :input_width, :])  # left bottom
    images.append(image[-input_height:, -input_width:, :])  # right bottom
    images.append(image[center_crop_y: center_crop_y + input_height, center_crop_x: center_crop_x + input_width, :])

    return np.array(images, dtype=np.float32)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 6.0 MB/s eta 0:00:0000:0100:01m
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(render

In [5]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model, models, Input, regularizers
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3, DenseNet121, VGG16



class VGG16Net(Model):
    def __init__(self):
        super(VGG16Net, self).__init__()
        self.base_model = VGG16(weights='imagenet', include_top=False, pooling='max')
        # self.base_model.trainable = False
        for i in range(len(self.base_model.layers) - 2):  # print(len(model.layers))=23
            self.base_model.layers[i].trainable = False
        self.d1 = Dense(units=hash_bit, activation=tf.nn.sigmoid)
        self.d2 = Dense(units=num_class, activation=tf.nn.softmax)

    def call(self, x):
        x = self.base_model(x)
        hash_code = self.d1(x)
        prediction = self.d2(hash_code)
        return prediction, hash_code

class VGG19Net(Model):
    def __init__(self):
        super(VGG19Net, self).__init__()
        self.base_model = VGG19(weights='imagenet', include_top=False, pooling='max')
        # self.base_model.trainable = False
        #for i in range(len(self.base_model.layers)):
        #    self.base_model.layers[i].trainable = False
        self.d1 = Dense(units=hash_bit, activation=None)
        self.bn1 = BatchNormalization()
        self.ac1 = tf.keras.layers.Activation("sigmoid")
        self.d2 = Dense(units=num_class, activation=tf.nn.softmax)

    def call(self, x):
        x = self.base_model(x)
        hash_code = self.d1(x)
        hash_code = self.bn1(hash_code)
        hash_code = self.ac1(hash_code)
        prediction = self.d2(hash_code)
        return prediction, hash_code    


def Resnet_Model():
    base_model = ResNet50(weights='imagenet', include_top=False, layers=tf.keras.layers, input_shape=(224, 224, 3))
    print(len(base_model.layers))  # 175
    #for layer in base_model.layers[:170]:
    #    layer.trainable = False
    #for layer in base_model.layers[170:]:
    #    layer.trainable = True
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D(name='average_pool')(x)
    x = tf.keras.layers.Flatten(name='flatten')(x)
    hash_value = tf.keras.layers.Dense(units=hash_bit, activation=None, kernel_regularizer=regularizers.l2(0.001))(x)
    # hash_value = tf.keras.layers.Dense(units=hash_bit, activation=None)(x)
    hash_value = tf.keras.layers.BatchNormalization()(hash_value)
    hash_value = tf.keras.layers.Activation("sigmoid")(hash_value)
    prediction = tf.keras.layers.Dense(units=num_class, activation="softmax",
                                       kernel_regularizer=regularizers.l2(0.001))(hash_value)
    # prediction = tf.keras.layers.Dense(units=num_class, activation="softmax")(hash_value)
    model = models.Model(inputs=base_model.input, outputs=[prediction, hash_value])
    return model


def DenseNet_Model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    print(len(base_model.layers))  # 427
    for layer in base_model.layers[:415]:
       layer.trainable = False
    for layer in base_model.layers[415:]:
       layer.trainable = True
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D(name='average_pool')(x)
    x = tf.keras.layers.Flatten(name='flatten')(x)
    #x = tf.keras.layers.Dropout(0.5)(x)
    hash_value = tf.keras.layers.Dense(units=hash_bit, activation=None)(x)
    hash_value = tf.keras.layers.BatchNormalization()(hash_value)
    hash_value = tf.keras.layers.Activation("sigmoid")(hash_value)
    prediction = tf.keras.layers.Dense(units=num_class, activation="softmax")(hash_value)
    model = models.Model(inputs=base_model.input, outputs=[prediction, hash_value])
    return model




In [6]:

import numpy as np
import os
import cv2
tf.enable_eager_execution()




def load_list(list_path, image_root_path):
    images = []
    labels = []
    with open(list_path, 'r') as f:
        for line in f:
            lines = line.replace('\n', '').split('\t')
            if len(lines) == 1:
                lines = line.replace('\n', '').split()
            #print(lines)
            # images.append(os.path.join(image_root_path, line[0]))
            images.append(image_root_path + lines[0])
            labels.append(int(lines[1]))
    return images, labels


def load_image(image_path, label, augment=False, crop_10=False):
    """
    In training, it is highly recommended to set the augment to true.
    In test, the standard 10-crop test [1] is provided for fair comparison.
    [1] K. He, X. Zhang, S. Ren, and J. Sun. Deep residual learning for image recognition. In CVPR, 2016.
    """
    image = cv2.imread(image_path.numpy().decode()).astype(np.float32)

    if augment:
        image = random_aspect(image)
        image = random_size(image)
        image = random_crop(image)
        image = random_flip(image)
        image = random_hsv(image)
        image = random_pca(image)
    else:
        image = random_size(image, target_size=256)
        if crop_10:
            image = test_10_crop(image)
        else:
            image = center_crop(image)

    image = normalize(image)

    label_one_hot = np.zeros(num_class)
    label_one_hot[label] = 1.0

    return image, label_one_hot


def train_iterator(list_path=imagenet_train_path):
    # list_path图片名称，train_data_path图片前缀地址
    

    images, labels = load_list(list_path, train_data_path)
    #print(images, labels)
    #with open("image-labels.txt", 'w') as f:
    #        f.write("images"+str(images))
    #        f.write("\n")
    #        f.write("labels"+str(labels))
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(len(images))
    dataset = dataset.repeat()
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, True, False], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    it = dataset.__iter__()
    return it


def test_iterator(list_path=imagenet_test_path):
    images, labels = load_list(list_path, test_data_path)
    
    print("images:",len(images),"labels:",len(labels))
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, False, False], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    it = dataset.__iter__()
    return it


def test_10_crop_iterator(list_path=imagenet_test_path):
    images, labels = load_list(list_path, test_data_path)
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, False, True], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    it = dataset.__iter__()
    return it


# if __name__ == '__main__':
#     # Annotate the 'normalize' function for visualization
#     # it = train_iterator()
#     it = test_iterator('/content/test.txt')
#     images, labels = it.next()
#     # print(np.shape(images), np.shape(labels))
#     for i in range(10):
#         print(np.where(labels[i].numpy() != 0))
#         cv2_imshow( images[i].numpy().astype(np.uint8))
#         cv2.waitKey(0)


In [7]:

import numpy as np
import os
import cv2
tf.enable_eager_execution()




def load_list(list_path, image_root_path):
    images = []
    labels = []
    with open(list_path, 'r') as f:
        for line in f:
            lines = line.replace('\n', '').split('\t')
            if len(lines) == 1:
                lines = line.replace('\n', '').split()
            #print(lines)
            # images.append(os.path.join(image_root_path, line[0]))
            images.append(image_root_path + lines[0])
            labels.append(int(lines[1]))
    return images, labels


def load_image(image_path, label, augment=False, crop_10=False):
    """
    In training, it is highly recommended to set the augment to true.
    In test, the standard 10-crop test [1] is provided for fair comparison.
    [1] K. He, X. Zhang, S. Ren, and J. Sun. Deep residual learning for image recognition. In CVPR, 2016.
    """
    image = cv2.imread(image_path.numpy().decode()).astype(np.float32)

    if augment:
        image = random_aspect(image)
        image = random_size(image)
        image = random_crop(image)
        image = random_flip(image)
        image = random_hsv(image)
        image = random_pca(image)
    else:
        image = random_size(image, target_size=256)
        if crop_10:
            image = test_10_crop(image)
        else:
            image = center_crop(image)

    image = normalize(image)

    label_one_hot = np.zeros(num_class)
    label_one_hot[label] = 1.0

    return image, label_one_hot


def train_iterator(list_path=imagenet_train_path):
    # list_path图片名称，train_data_path图片前缀地址
    

    images, labels = load_list(list_path, train_data_path)
    #print(images, labels)
    #with open("image-labels.txt", 'w') as f:
    #        f.write("images"+str(images))
    #        f.write("\n")
    #        f.write("labels"+str(labels))
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(len(images))
    dataset = dataset.repeat()
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, True, False], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    it = dataset.__iter__()
    return it


def test_iterator(list_path=imagenet_test_path):
    images, labels = load_list(list_path, test_data_path)
    
    print("images:",len(images),"labels:",len(labels))
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, False, False], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    it = dataset.__iter__()
    return it


def test_10_crop_iterator(list_path=imagenet_test_path):
    images, labels = load_list(list_path, test_data_path)
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(
        lambda x, y: tf.py_function(load_image, inp=[x, y, False, True], Tout=[tf.float32, tf.float32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    it = dataset.__iter__()
    return it


# if __name__ == '__main__':
#     # Annotate the 'normalize' function for visualization
#     # it = train_iterator()
#     it = test_iterator('/content/test.txt')
#     images, labels = it.next()
#     # print(np.shape(images), np.shape(labels))
#     for i in range(10):
#         print(np.where(labels[i].numpy() != 0))
#         cv2_imshow( images[i].numpy().astype(np.uint8))
#         cv2.waitKey(0)


In [ ]:
#!/usr/bin/python?
# -*- coding: utf-8 -*-import tensorflow as tf
import numpy as np
import pickle as p
from tqdm import tqdm
import os
import cv2
from tensorflow.keras import models, optimizers, Sequential
from tensorflow.keras.layers import Input

# from load_data import train_iterator, test_iterator
import tensorflow as tf

#import datetime
#current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# log_dir = 'logs/Resnet/' + "cos"
# summary_writer = tf.summary.create_file_writer(log_dir) 
plot_train_loss=[]
plot_test_loss=[]
plot_train_acc=[]
plot_test_acc=[]
class CosineDecayWithWarmUP(tf.keras.experimental.CosineDecay):
    def __init__(self, initial_learning_rate, decay_steps, alpha=0.0, warm_up_step=0, name=None):
        self.warm_up_step = warm_up_step
        super(CosineDecayWithWarmUP, self).__init__(initial_learning_rate=initial_learning_rate,
                                                    decay_steps=decay_steps,
                                                    alpha=alpha,
                                                    name=name)

def hash_loss_fn(hash_input):
    loss1 = -1 * tf.reduce_mean(tf.square(hash_input - 0.5)) + 0.25  # 最大值为0.25
    loss2 = tf.reduce_mean(tf.square(tf.reduce_mean(hash_input, axis=1) - 0.5))
    return loss1 + loss2


def cross_entropy(y_true, y_pred):
    cross_entropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return tf.reduce_mean(cross_entropy)


def l2_loss(model, weights=weight_decay):
    variable_list = []
    for v in model.trainable_variables:
        if 'kernel' in v.name:
            variable_list.append(tf.nn.l2_loss(v))
    return tf.add_n(variable_list) * weights


def accuracy(y_true, y_pred):
    correct_num = tf.equal(tf.argmax(y_true, -1), tf.argmax(y_pred, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_num, dtype=tf.float32))
    return accuracy


@tf.function
def train_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        prediction, hash_value = model(x, training=True)  # 修改
        hash_loss = hash_loss_fn(hash_value)  # 修改
        ce = cross_entropy(y, prediction)
        l2 = l2_loss(model)
        loss = ce + l2 + hash_loss  # 修改
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return ce, hash_loss, prediction


@tf.function
def test_step(model, x, y):
    prediction, _ = model(x, training=False)  # 修改
    ce = cross_entropy(y, prediction)
    return ce, prediction


def train(model, optimizer, train_iterator, log_file):
    sum_loss = 0
    sum_hash_loss = 0
    sum_accuracy = 0

    for i in tqdm(range(iterations_per_epoch)):
        x, y = train_iterator.next()
        loss, hash_loss, prediction = train_step(model, optimizer, x, y)
        sum_hash_loss += hash_loss
        sum_loss += loss
        sum_accuracy += accuracy(y, prediction)

    print('ce_loss:%f, l2_loss:%f,hash_loss:%f ,accuracy:%f' %
          (sum_loss / iterations_per_epoch, l2_loss(model), sum_hash_loss / iterations_per_epoch,
           sum_accuracy / iterations_per_epoch))
    log_file.write('train: ce loss: {:.4f}, l2 loss: {:.4f}, hash_loss: {:.4f}, accuracy: {:.4f}\n'.format(sum_loss / iterations_per_epoch, l2_loss(model), 
    sum_hash_loss / iterations_per_epoch, sum_accuracy / iterations_per_epoch ))
    plot_train_loss.append(sum_loss/iterations_per_epoch)
    plot_train_loss.append(sum_accuracy/iterations_per_epoch)


def test(model, test_iter, log_file):
    sum_loss = 0
    sum_accuracy = 0
    test_data_iterator = test_iterator()

    for i in tqdm(range(test_iter)):
        x, y = test_data_iterator.next()
        x = tf.cast(x, tf.float32)  # 将测试集中的图像编码成float32
        loss, prediction = test_step(model, x, y)
        sum_loss += loss
        sum_accuracy += accuracy(y, prediction)
    print('test, loss:%f, accuracy:%f' %
          (sum_loss / test_iterations, sum_accuracy / test_iterations))
    log_file.write('test: loss: {:.4f}, accuracy: {:.4f}\n'.format(sum_loss / test_iterations, sum_accuracy / test_iterations))
    plot_test_loss.append(sum_loss/test_iterations)
    plot_test_loss.append(sum_accuracy/test_iterations)
    return sum_accuracy / test_iterations


if __name__ == '__main__':
    # gpu config
    gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    import os

    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    physical_devices = tf.config.experimental.list_physical_devices(device_type='GPU')
    print(physical_devices)

    tf.config.experimental.set_memory_growth(device=physical_devices[0], enable=True)

    # show
    model = VGG19Net()
    model.build((None,224,224,3))
    model.summary()

    # train-cos
    learning_rate_schedules = CosineDecayWithWarmUP(initial_learning_rate=initial_learning_rate,
                                                decay_steps=epoch_num * iterations_per_epoch - warm_iterations,
                                                alpha=minimum_learning_rate,
                                                warm_up_step=warm_iterations)
    optimizer = optimizers.SGD(learning_rate=learning_rate_schedules, momentum=0.9)
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)
    
    # train
    #learning_rate_schedules = optimizers.schedules.PiecewiseConstantDecay(c.boundaries, c.learning_rate)
    #optimizer = optimizers.SGD(learning_rate=learning_rate_schedules, momentum=0.9, nesterov=True)
    
    # optimizer = optimizers.Adam(learning_rate=learning_rate_schedules)
    # optimizer = optimizers.Adam()
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)

    train_data_iterator = train_iterator()
    for epoch in range(epoch_num):
        print(epoch)
        with open(log_file, 'a') as f:
            f.write('epoch:{}\n'.format(epoch))
            train(model, optimizer, train_data_iterator, f)
            test_acc = test(model, test_iterations, f)
        if test_acc > 0.5:
            name = "./checkpoint/vgg_new/" + str(np.round(test_acc.numpy(), 3)) + '/model.ckpt'
            if not os.path.exists("./checkpoint/vgg_new/" + str(np.round(test_acc.numpy(), 3))):
                os.makedirs("./checkpoint/vgg_new/" + str(np.round(test_acc.numpy(), 3)))
            checkpoint.save(name)

2024-05-02 19:30:43.221653: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-05-02 19:30:43.323234: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA A100 80GB PCIe major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:4b:00.0
2024-05-02 19:30:43.323317: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-05-02 19:30:43.370872: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-05-02 19:30:43.422102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2024-05-02 19:30:43.422540: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2024-05-02 19:30:43.439434: I tensorflow/stream_executor/platform/de

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-05-02 19:30:43.507207: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2100000000 Hz
2024-05-02 19:30:43.510459: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4f38390 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-02 19:30:43.510502: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-05-02 19:30:43.699148: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2596590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-02 19:30:43.699210: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-05-02 19:30:43.700744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA A100 80GB PCIe major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:4b:00.0
2024-05-02 1

80142336/80134624 [==============================] - 12s 0us/step
Model: "vg_g19net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 512)               20024384  
_________________________________________________________________
dense (Dense)                multiple                  262656    
_________________________________________________________________
batch_normalization (BatchNo multiple                  2048      
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  246240    
Total params: 20,535,328
Trainable params: 20,534,304
Non-trainable params: 1,024
_________________________________________________________________
0


  0%|                                                                                                | 0/1236 [00:00<?, ?it/s]2024-05-02 19:31:02.747467: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-05-02 19:31:03.640574: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
100%|█████████████████████████████████████████████████████████████████████████████████████| 1236/1236 [12:41<00:00,  1.62it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████| 309/309 [02:05<00:00,  2.46it/s]


test, loss:0.367536, accuracy:0.914042
25


100%|█████████████████████████████████████████████████████████████████████████████████████| 1236/1236 [09:44<00:00,  2.11it/s]


ce_loss:0.325121, l2_loss:0.531800,hash_loss:0.099165 ,accuracy:0.927551
images: 39539 labels: 39539


 50%|███████████████████████████████████████████▎                                           | 154/309 [01:04<01:59,  1.30it/s]

In [ ]:
import matplotlib.pyplot as plt
pochs = range(epoch_num)
print(len(plot_train_loss))
kkk=0
plot_ac_train_loss=[]
plot_ac_test_loss=[]
for i in plot_train_loss:
    if kkk==0:
        plot_ac_train_loss.append(i)
        kkk=1
    else:
        kkk=0

kkk=0
for i in plot_test_loss:
    if kkk==0:
        plot_ac_test_loss.append(i)
        kkk=1
    else:
        kkk=0
plt.plot(pochs, plot_ac_train_loss, label='Train Loss')

plt.plot(pochs, plot_ac_test_loss, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Testing Loss')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
pochs = range(epoch_num)

kkk=1
plot_ac_train_acc=[]
plot_ac_test_acc=[]
for i in plot_train_loss:
    if kkk==0:
        plot_ac_train_acc.append(i)
        kkk=1
    else:
        kkk=0

kkk=1
for i in plot_test_loss:
    if kkk==0:
        plot_ac_test_acc.append(i)
        kkk=1
    else:
        kkk=0
print(len(plot_test_acc))    
plt.plot(pochs, plot_ac_train_acc, label='Train Accuracy')
plt.plot(pochs, plot_ac_test_acc, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Testing Accuracy')
plt.legend()